In [5]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

import base64
import urllib.parse

# Data / plotting
import pandas as pd

# CRUD module
from CRUD_Python_Module import AnimalShelter

JupyterDash.infer_jupyter_proxy_config()

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "WRITE_THIS_PASSWORD_DOWN"   

db = AnimalShelter(user=username, password=password)

# Load ALL data initially
df = pd.DataFrame.from_records(db.read({}))

# Remove _id to prevent DataTable crash
if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)

#########################
# Dashboard Layout / View
#########################

app = JupyterDash(__name__)

# ---- Logo (base64) ----
image_filename = "Grazioso Salvare Logo.png"  
encoded_image = base64.b64encode(open(image_filename, "rb").read()).decode()

app.layout = html.Div([
    # Logo + Title + Identifier
    html.Center([
        html.Img(
            src=f"data:image/png;base64,{encoded_image}",
            style={"height": "150px", "marginBottom": "10px"}
        ),
        html.H1("CS-340 Dashboard"),
        html.H3("Rafael Salazar | Grazioso Salvare", style={"marginTop": "0px"})
    ]),

    html.Hr(),

    # ---- FILTER OPTIONS (Controller input) ----
    html.Div([
        html.H4("Rescue Filter"),
        dcc.RadioItems(
            id="filter-type",
            options=[
                {"label": "Water Rescue", "value": "water"},
                {"label": "Mountain/Wilderness Rescue", "value": "mountain"},
                {"label": "Disaster/Tracking", "value": "disaster"},
                {"label": "Reset", "value": "reset"}
            ],
            value="reset",
            inline=True
        )
    ]),

    html.Hr(),

    # ---- DATA TABLE ----
    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict("records"),

        # required for map callback
        row_selectable="single",
        selected_rows=[0],

        # usability
        page_size=10,
        sort_action="native",
        filter_action="native",
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left", "padding": "5px"}
    ),

    html.Br(),
    html.Hr(),

    # ---- CHART + MAP side-by-side ----
    html.Div(
        className="row",
        style={"display": "flex"},
        children=[
            html.Div(id="graph-id", className="col s12 m6"),
            html.Div(id="map-id", className="col s12 m6")
        ]
    )
])

#############################################
# Controller Callbacks
#############################################

# 1) Filter table based on rescue type
@app.callback(
    Output("datatable-id", "data"),
    Input("filter-type", "value")
)
def update_dashboard(filter_type):

    if filter_type == "water":
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }

    elif filter_type == "mountain":
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }

    elif filter_type == "disaster":
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }

    else:
        query = {}

    dff = pd.DataFrame.from_records(db.read(query))
    if "_id" in dff.columns:
        dff.drop(columns=["_id"], inplace=True)

    return dff.to_dict("records")


# 2) Pie chart updates based on table view
@app.callback(
    Output("graph-id", "children"),
    Input("datatable-id", "derived_virtual_data")
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.Div("No data to display.")

    dff = pd.DataFrame(viewData)

    if "breed" not in dff.columns:
        return html.Div("Breed column not found.")

    fig = px.pie(dff, names="breed", title="Breed Distribution (Filtered View)")
    return [dcc.Graph(figure=fig)]


# 3) Highlight selected columns 
@app.callback(
    Output("datatable-id", "style_data_conditional"),
    Input("datatable-id", "selected_columns")
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [
        {"if": {"column_id": i}, "backgroundColor": "#D2F3FF"}
        for i in selected_columns
    ]


# 4) Map updates based on selected row
@app.callback(
    Output("map-id", "children"),
    [Input("datatable-id", "derived_virtual_data"),
     Input("datatable-id", "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or len(viewData) == 0:
        return []

    dff = pd.DataFrame(viewData)
    row = 0 if not index else index[0]


    if "location_lat" not in dff.columns or "location_long" not in dff.columns:
        return html.Div("Map error: location_lat/location_long columns not found.")

    lat = dff.iloc[row]["location_lat"]
    lon = dff.iloc[row]["location_long"]

    breed = dff.iloc[row]["breed"] if "breed" in dff.columns else "Unknown breed"
    name = dff.iloc[row]["name"] if "name" in dff.columns else "Unknown name"

    return [
        dl.Map(
            style={"width": "1000px", "height": "500px"},
            center=[30.75, -97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(breed),
                        dl.Popup([html.H1("Animal Name"), html.P(name)])
                    ]
                )
            ]
        )
    ]


# Run the app
app.run_server()

Dash app running on https://monitoroberon-jetprelude-3000.codio.io/proxy/8050/
